# Twelve Labs Eddie

- make sure to `git clone https://github.com/twelvelabs-io/tl-jockey.git`
- then switch to `pumplerod-hotfix` branch
- - `git checkout -b pumplerod-hotfix`
- - `git pull --tags origin pumplerod-hotfix`

In [ ]:
import os
import sys
import random
import ffmpeg
import asyncio
from IPython.display import Video
#from dotenv import load_dotenv

In [ ]:
from twelvelabs import TwelveLabs, APIStatusError
from twelvelabs.models.task import Task
#import dropbox

In [ ]:
from jockey.util import *
from jockey.stirrups.video_editing import *

In [ ]:
# load_dotenv()

In [ ]:
SOURCE_VIDEO_PATH = "/Project/12Labs/hackathon/video/source"
OUTPUT_VIDEO_PATH = "/Project/12Labs/hackathon/video/output"
JOCKEY_VIDEO_PATH = '/Project/12Labs/hackathon/jockey_public_dir'
JOCKEY_DB_PATH = '/Project/12Labs/hackathon/jockey_vector_db_dir'
TL_API_KEY = 'tlk_3FFX0ZW3XY8HMQ22Q7A3S028TTTR'
INDEX_ID = '6664a657913827e5316d503b'

In [ ]:
os.environ[ "TWELVE_LABS_API_KEY"] = TL_API_KEY
os.environ[ "HOST_PUBLIC_DIR"] = JOCKEY_VIDEO_PATH
os.environ[ "HOST_VECTOR_DB_DIR"] = JOCKEY_DB_PATH

# Helper Functions

In [ ]:
def parse_search_result( page):
    output = [{'video_id':clip.video_id, 'score':clip.score, 'start':clip.start, 'end':clip.end} for clip in page]
    return output

In [ ]:
async def generate_sequence( joined_group, output):
    #ffmpeg.output( joined_group[0], joined_group[1], output).overwrite_output().run_async()
    ffmpeg.output( joined_group[0], joined_group[1], output).run_async()


In [ ]:
def print_page(page):
  for clip in page:
    print(
        f" video_id={clip.video_id} score={clip.score} start={clip.start} end={clip.end} confidence={clip.confidence}"
    )

# TwelveLabs Calls

In [ ]:
#client = TwelveLabs(api_key=os.getenv('TL_API_KEY'))
client = TwelveLabs(api_key=TL_API_KEY)

In [ ]:
cut_list = [ "Drone shots of mountains",
             "Johnny walking among massive mountains",
             "military historian B roll. Johnny showing me his sick 3d map and maybing saying something about it."]

In [ ]:
clips_dicts = []
for clip_bundle in cut_list:
    print( f'Looking for: {clip_bundle}')
    search_results = client.search.query(index_id=INDEX_ID, query=clip_bundle, options=["visual"])
    bundle_results = parse_search_result(search_results.data)
    clips_dicts.append( bundle_results)
    #break

In [ ]:
clips_dicts

In [ ]:
num_sequences = 3
output_files = []
for seq_n in range(num_sequences):
    sequence_clips = []
    for c_dict in clips_dicts:
        select_idx = random.randint(0, len(c_dict)-1)
        file_path = download_video(video_id=c_dict[ select_idx]['video_id'], index_id=INDEX_ID, start=c_dict[ select_idx]['start'], end=c_dict[ select_idx]['end'])
        print( file_path)
        clip = {'video_id':c_dict[ select_idx]['video_id'], 
                'start':c_dict[ select_idx]['start'], 
                'end':c_dict[ select_idx]['end'], 
                'video_filepath':file_path}
        sequence_clips.append( clip)

    out_file = f'{OUTPUT_VIDEO_PATH}/sequence_{seq_n + 1}.mp4'
    output_files.append( out_file)
    await combine_clips( clips=sequence_clips, output_filename=out_file, index_id=INDEX_ID)

In [ ]:
    input_streams = []
    output_filename=f'{OUTPUT_VIDEO_PATH}/sequence_{seq_n + 1}.mp4'
    index_id = INDEX_ID
    for clip in sequence_clips:
        video_id = clip['video_id']
        start = clip['start']
        end = clip['end']
        video_filepath = os.path.join(os.environ["HOST_PUBLIC_DIR"], index_id, f"{video_id}_{start}_{end}.mp4")
        print( f'adding file: {video_filepath}')
        '''
        if os.path.isfile(video_filepath) is False:
            try:
                download_video(video_id=video_id, index_id=index_id, start=start, end=end)
            except AssertionError as error:
                error_response = {
                    "message": f"There was an error retrieving the video metadata for Video ID: {video_id} in Index ID: {index_id}. "
                    "Double check that the Video ID and Index ID are valid and correct.",
                    "error": str(error)
                }
                return error_response
        '''
        clip_video_input_stream = ffmpeg.input(filename=video_filepath, loglevel="verbose").video
        clip_audio_input_stream = ffmpeg.input(filename=video_filepath, loglevel="verbose").audio
        clip_video_input_stream = clip_video_input_stream.filter("setpts", "PTS-STARTPTS")
        clip_audio_input_stream = clip_audio_input_stream.filter("asetpts", "PTS-STARTPTS")
        
        input_streams.append(clip_video_input_stream)
        input_streams.append(clip_audio_input_stream)
    
    output_filepath = os.path.join(os.environ["HOST_PUBLIC_DIR"], index_id, output_filename)
    ffmpeg.concat(*input_streams, v=1, a=1).output(output_filepath, acodec="libmp3lame", loglevel="verbose").overwrite_output().run()
    '''

In [ ]:
sequence_clips

In [ ]:
combine_clips( clips=sequence_clips, output_filename=f'{OUTPUT_VIDEO_PATH}/sequence_test.mp4', index_id=INDEX_ID)
#combine_clips( sequence_clips, f'{OUTPUT_VIDEO_PATH}/sequence_test.mp4', INDEX_ID)

In [ ]:
input_streams = []
output_filename=f'{OUTPUT_VIDEO_PATH}/sequence_test.mp4'
index_id = INDEX_ID
for clip in sequence_clips:
    print( clip)
    video_id = clip['video_id']
    start = clip['start']
    end = clip['end']
    video_filepath = os.path.join(os.environ["HOST_PUBLIC_DIR"], index_id, f"{video_id}_{start}_{end}.mp4")
    print( f'adding file: {video_filepath}')
    '''
    if os.path.isfile(video_filepath) is False:
        try:
            download_video(video_id=video_id, index_id=index_id, start=start, end=end)
        except AssertionError as error:
            error_response = {
                "message": f"There was an error retrieving the video metadata for Video ID: {video_id} in Index ID: {index_id}. "
                "Double check that the Video ID and Index ID are valid and correct.",
                "error": str(error)
            }
            return error_response
    '''
    clip_video_input_stream = ffmpeg.input(filename=video_filepath, loglevel="verbose").video
    clip_audio_input_stream = ffmpeg.input(filename=video_filepath, loglevel="verbose").audio
    clip_video_input_stream = clip_video_input_stream.filter("setpts", "PTS-STARTPTS")
    clip_audio_input_stream = clip_audio_input_stream.filter("asetpts", "PTS-STARTPTS")
    
    input_streams.append(clip_video_input_stream)
    input_streams.append(clip_audio_input_stream)

output_filepath = os.path.join(os.environ["HOST_PUBLIC_DIR"], index_id, output_filename)
ffmpeg.concat(*input_streams, v=1, a=1).output(output_filepath, acodec="libmp3lame", loglevel="verbose").overwrite_output().run()
    

In [ ]:
import pprint
pprint.pprint( f'Generated: {output_files}')


In [ ]:
print( output_files[0])
Video( output_files[0], embed=True)


In [ ]:
print( output_files[1])
Video( output_files[1], embed=True)


In [ ]:
print( output_files[2])
Video( output_files[2], embed=True)
